###  CPA xlsx to Tidydata

In [1]:
from databaker.framework import *

In [2]:
import pandas as pd 

In [3]:
inputFile = 'mq10.xlsx'

In [4]:
tab = loadxlstabs(inputFile)

Loading mq10.xlsx which has size 2222819 bytes
Table names: ['data']


In [5]:
tab = tab[0]

In [6]:
observations = tab.excel_ref('B8').expand(DOWN).expand(RIGHT).is_not_blank()

In [7]:
Title = tab.excel_ref('B1').expand(RIGHT).is_not_blank()

In [8]:
CDID = tab.excel_ref('B2').expand(RIGHT).is_not_blank()

In [9]:
Year = tab.excel_ref('A8').expand(DOWN).is_not_blank()

In [10]:
Currency = tab.excel_ref('P3')

In [18]:
Dimensions = [
            HDimConst('Geography', 'K02000001'),
            HDim(Year,'TIME',DIRECTLY,LEFT),
            HDim(CDID,'Cdid',DIRECTLY,ABOVE),
            HDimConst('Unit', '£ Millions'), 
            HDimConst('Measure Type','GBP Total'),
            HDim( Title, 'Title', DIRECTLY, ABOVE)
]

In [19]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)

In [20]:
new_table = c1.topandas()

multiple TIMEUNITs: 'Year'(87103), 'Quarter'(348477)


In [21]:
print(len(new_table))

435580


In [22]:
temp_table = pd.read_excel('cord_sitc classification table.xlsx', sheet_name = 0)

In [23]:
temp_table = temp_table.reset_index()

In [24]:
temp_table.drop(['level_0','level_1','Sequence'], axis =1, inplace = True)

In [25]:
temp_table.tail()

,cdid,COMMODITY,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
6196,BPFR,5min8minE,WW,IM,BOP,IDEF,NSA,M
6197,BPGR,5min8minE,WW,IM,BOP,IDEF,SA,M
6198,BPCR,5min8minE,WW,IM,BOP,VM,NSA,M
6199,ELAI,5min8minE,WW,IM,BOP,VM,SA,M
6200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
classification1 = pd.read_excel('CSDB classification tables.xlsx', 'cord_sitc')

In [27]:
classification1.tail(5)

,cdid,COMMODITY,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
6193,ELAX,5min8minE,WW,IM,BOP,DEF,NSA,Q
6194,BPFR,5min8minE,WW,IM,BOP,IDEF,NSA,Q
6195,BPGR,5min8minE,WW,IM,BOP,IDEF,SA,Q
6196,BPCR,5min8minE,WW,IM,BOP,VM,NSA,Q
6197,ELAI,5min8minE,WW,IM,BOP,VM,SA,Q


In [28]:
classification2 = pd.read_excel('CSDB classification tables.xlsx', 'cord_cpa')

In [29]:
classification2.head(5)

,cdid,PRODUCT,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
0,P42L,24.2,EU,EX,BOP,CP,NSA,Q
1,P483,24.2,EU,EX,BOP,CP,SA,Q
2,P4DJ,24.2,EU,EX,BOP,CVM,NSA,Q
3,P4IZ,24.2,EU,EX,BOP,CVM,SA,Q
4,P3EP,24.2,EU,IM,BOP,CP,NSA,Q


In [30]:
classification2.rename(index = str, columns = {'PRODUCT':'COMMODITY'}, inplace = True)

In [31]:
classification3 = pd.read_excel('CSDB classification tables.xlsx', 'cord_country')

In [32]:
classification3.head(5)

,cdid,COUNTRY,DIRECTION,BASIS,SEASADJ,PERIOD
0,KN2O,XS,BAL,BOP,NSA,Q
1,LGDS,V4,BAL,BOP,NSA,Q
2,L87P,V3,BAL,BOP,NSA,Q
3,L87J,V2,BAL,BOP,NSA,Q
4,MHN8,I7,BAL,BOP,NSA,Q


In [33]:
classification3['COMMODITY'] = ''
classification3['PRICE'] = ''

In [34]:
classification3.rename(index = str, columns = {'COUNTRY':'AREA'}, inplace = True)

In [35]:
temp_table = pd.concat([temp_table, classification1, classification2,classification3])

In [36]:
temp_table.head()

,AREA,BASIS,COMMODITY,DIRECTION,PERIOD,PRICE,SEASADJ,cdid
0,EU,BOP,2plus4,BAL,M,CP,NSA,SDSX
1,EU,BOP,5minus8,BAL,M,CP,NSA,SGLO
2,EU,BOP,7plus8,BAL,M,CP,NSA,SFJC
3,EU,BOP,TminusO,BAL,M,CP,NSA,LKTX
4,EU,BOP,0plus1,BAL,M,CP,NSA,SDMS


In [37]:
temp_table.shape

(18264, 8)

In [38]:
temp_table.drop_duplicates(['cdid'], keep='first', inplace=True)

In [39]:
temp_table.shape

(12043, 8)

In [40]:
new_table.head()

,OBS,TIME,TIMEUNIT,Geography,Cdid,Unit,Measure Type,Title
0,16484.0,1998,Year,K02000001,P25X,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
1,103.0,1998,Year,K02000001,P269,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 30.4. Military fighti...
2,22961.0,1998,Year,K02000001,P25W,£ Millions,GBP Total,"CPA 08:WW:IM:CP:BOP:NSA: 29. Motor vehicles, t..."
3,247.0,1998,Year,K02000001,P268,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 30.2. Railway locomot...
4,6165.0,1998,Year,K02000001,P25Z,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 29.3. Parts & accesso...


In [ ]:
# new_table.head(1)['Cdid']

In [ ]:
# temp_table.head(1)['cdid']

In [41]:
new_table = pd.merge(new_table,temp_table, how = 'left', left_on = 'Cdid', right_on = 'cdid')

In [42]:
new_table.head(5)

,OBS,TIME,TIMEUNIT,Geography,Cdid,Unit,Measure Type,Title,AREA,BASIS,COMMODITY,DIRECTION,PERIOD,PRICE,SEASADJ,cdid
0,16484.0,1998,Year,K02000001,P25X,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,WW,BOP,29.1,IM,Q,CP,NSA,P25X
1,103.0,1998,Year,K02000001,P269,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 30.4. Military fighti...,WW,BOP,30.4,IM,Q,CP,NSA,P269
2,22961.0,1998,Year,K02000001,P25W,£ Millions,GBP Total,"CPA 08:WW:IM:CP:BOP:NSA: 29. Motor vehicles, t...",WW,BOP,29,IM,Q,CP,NSA,P25W
3,247.0,1998,Year,K02000001,P268,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 30.2. Railway locomot...,WW,BOP,30.2,IM,Q,CP,NSA,P268
4,6165.0,1998,Year,K02000001,P25Z,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 29.3. Parts & accesso...,WW,BOP,29.3,IM,Q,CP,NSA,P25Z


In [43]:
new_table.drop(['cdid'], axis = 1, inplace = True)

In [44]:
new_table['Year'] = new_table['TIME'].map(lambda x: str(x)[:4])

In [45]:
new_table['Months/Quarter'] = new_table['TIME'].map(lambda x: str(x)[4:])

In [46]:
new_table = new_table.drop('TIME', axis=1)

In [47]:
new_table.rename(index= str, columns= {'Year':'TIME'}, inplace = True)

In [48]:
new_table.head(5)

,OBS,TIMEUNIT,Geography,Cdid,Unit,Measure Type,Title,AREA,BASIS,COMMODITY,DIRECTION,PERIOD,PRICE,SEASADJ,TIME,Months/Quarter
0,16484.0,Year,K02000001,P25X,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,WW,BOP,29.1,IM,Q,CP,NSA,1998,
1,103.0,Year,K02000001,P269,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 30.4. Military fighti...,WW,BOP,30.4,IM,Q,CP,NSA,1998,
2,22961.0,Year,K02000001,P25W,£ Millions,GBP Total,"CPA 08:WW:IM:CP:BOP:NSA: 29. Motor vehicles, t...",WW,BOP,29,IM,Q,CP,NSA,1998,
3,247.0,Year,K02000001,P268,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 30.2. Railway locomot...,WW,BOP,30.2,IM,Q,CP,NSA,1998,
4,6165.0,Year,K02000001,P25Z,£ Millions,GBP Total,CPA 08:WW:IM:CP:BOP:NSA: 29.3. Parts & accesso...,WW,BOP,29.3,IM,Q,CP,NSA,1998,


In [ ]:
# temp_table1 = new_table[new_table['AREA'].isnull() == True] 

In [ ]:
# temp_table1

In [ ]:
# Codelist = temp_table1['Cdid'].unique()

In [ ]:
# Codelist

In [ ]:
# codes = pd.Series(Codelist)

In [ ]:
# codes.to_csv('Codelist.csv', index = False)

In [49]:
new_table['Months/Quarter'] = new_table['Months/Quarter'].str.lstrip('')

In [50]:
new_table['Period'] = new_table['TIMEUNIT'].map(str)+'/'+new_table['TIME']+'-'+new_table['Months/Quarter']

In [51]:
new_table['Period'] = new_table['Period'].str.rstrip('-')

In [52]:
new_table['Period'] = new_table['Period'].map(lambda cell:cell.replace(' ', ''))

In [53]:
new_table.drop(['TIME'], axis = 1, inplace = True)
new_table.drop(['TIMEUNIT'], axis = 1, inplace = True)
new_table.drop(['Geography'], axis = 1, inplace = True)
new_table.drop(['PERIOD'], axis = 1, inplace = True)
new_table.drop(['Months/Quarter'], axis = 1, inplace = True)


In [54]:
new_table.rename(index= str, columns= {'OBS':'Value'}, inplace = True)
new_table.rename(index= str, columns= {'DIRECTION':'Flow'}, inplace = True)

In [81]:
new_table =new_table[['AREA','Period','Cdid','BASIS','COMMODITY','SEASADJ','Flow','PRICE','Measure Type','Value','Unit']]

In [82]:
new_table.tail(5)

,AREA,Period,Cdid,BASIS,COMMODITY,SEASADJ,Flow,PRICE,Measure Type,Value,Unit
435575,WW,Quarter/2017-Q4,P23O,BOP,15.2,NSA,IM,CP,GBP Total,1307.0,£ Millions
435576,WW,Quarter/2017-Q4,P23J,BOP,14.1,NSA,IM,CP,GBP Total,4095.0,£ Millions
435577,WW,Quarter/2017-Q4,P23I,BOP,14,NSA,IM,CP,GBP Total,5092.0,£ Millions
435578,WW,Quarter/2017-Q4,P23L,BOP,14.3,NSA,IM,CP,GBP Total,979.0,£ Millions
435579,WW,Quarter/2017-Q4,P23K,BOP,14.2,NSA,IM,CP,GBP Total,18.0,£ Millions


In [83]:
new_table.dtypes

AREA             object
Period           object
Cdid             object
BASIS            object
COMMODITY        object
SEASADJ          object
Flow             object
PRICE            object
Measure Type     object
Value           float64
Unit             object
dtype: object

In [84]:
new_table.to_csv('C:\\Users\\vamsi\\Desktop\\Gdrive\\Out\\Mret\\CPA_Tidydata.csv', index = False)